#**Updated Code for AAPL Stock prices dataset**

###**After obtaining data using quantrocket**

In [ ]:
import pandas as pd
prices = pd.read_csv("usstock_free_1d.csv", parse_dates=["Date"])
prices.head()

,Sid,Date,Close
0,FIBBG000B9XRY4,2023-01-03,124.2163
1,FIBBG000B9XRY4,2023-01-04,125.4975
2,FIBBG000B9XRY4,2023-01-05,124.1666
3,FIBBG000B9XRY4,2023-01-06,128.7352
4,FIBBG000B9XRY4,2023-01-09,129.2616


###**The Maximized portfolio value was calculated based on the transition probabilities and optimal buy indices**

In [35]:
class AAPL_Stocks:
    def __init__(self):
        self.value = 0
        self.state = 0  #Initially the state is set to Flat
        self.TransitionMatrix = {
            -1: {-1: 0, 0: 0, 1: 0},
            0: {-1: 0, 0: 0, 1: 0},
            1: {-1: 0, 0: 0, 1: 0}
        }

    def returnsPercent(self, current_price, prev_price):
        return (current_price - prev_price) / prev_price

    def PortfolioValue(self, stock, prev_state, current_state):
        if prev_state == 0 and current_state == 1:  # If bought the stock update the portfolio value
            self.value += 1
        elif prev_state == 0 and current_state == -1:
            self.value -= 1

    def update_TransitionMatrix(self, prev_state, current_state):
        self.TransitionMatrix[prev_state][current_state] += 1

    # Calculation of transition probabilities for transition matrix
    def TransitionProbabilities(self):
        trans_prob = {}
        for prev_state, transitions in self.TransitionMatrix.items():
            total_transitions = sum(transitions.values())
            if total_transitions == 0:
                continue
            trans_prob[prev_state] = { next_state: count / total_transitions for next_state, count in transitions.items()}
        return trans_prob

    # Making a decision to buy stock or not. If bought the buy will be on optimal buy indices
    def buy_stock_or_not(self, current_price, prev_price):
        returns = self.returnsPercent(current_price, prev_price)

        # 1 - Bull state , 0 - Flat state , -1 - Bear state
        if returns >= 0.01:
            current_state = 1
        elif -0.01 < returns < 0.01:
            current_state = 0
        else:
            current_state = -1

        prev_state = self.state
        # Update TransitionMatrix and Transition probabilities
        self.update_TransitionMatrix(prev_state, current_state)
        self.state = current_state

        # Calculate transition probabilities
        trans_prob = self.TransitionProbabilities()
        print("Matrix:")
        for key in sorted(trans_prob.keys()):
            print(f"{key}: {trans_prob[key]}")

        # Whether to buy the stock based on transition probabilities
        if self.state in trans_prob:
            additiveState = trans_prob[self.state].get(1, 0)
            reductiveState = trans_prob[self.state].get(-1, 0)
            if additiveState > reductiveState:
                stock = 1
                self.PortfolioValue(stock, prev_state, current_state)
            elif additiveState < reductiveState:
                stock = -1
            else:
                stock = 0
        else:
            stock = 0

        return stock


portfolio = AAPL_Stocks()


for i in range(1, len(prices['Close'])):
    stock = portfolio.buy_stock_or_not(prices['Close'][i], prices['Close'][i-1])
    decision_text = "Y" if stock == 1 else "N"
    print("-------------------------------------------------------------------------------------------")
    print(f"Day {i+1}: \nDecision: {decision_text} \nPortfolio Value: {portfolio.value}")


Matrix:
0: {-1: 0.0, 0: 0.0, 1: 1.0}
-------------------------------------------------------------------------------------------
Day 2: 
Decision: N 
Portfolio Value: 0
Matrix:
0: {-1: 0.0, 0: 0.0, 1: 1.0}
1: {-1: 1.0, 0: 0.0, 1: 0.0}
-------------------------------------------------------------------------------------------
Day 3: 
Decision: N 
Portfolio Value: 0
Matrix:
-1: {-1: 0.0, 0: 0.0, 1: 1.0}
0: {-1: 0.0, 0: 0.0, 1: 1.0}
1: {-1: 1.0, 0: 0.0, 1: 0.0}
-------------------------------------------------------------------------------------------
Day 4: 
Decision: N 
Portfolio Value: 0
Matrix:
-1: {-1: 0.0, 0: 0.0, 1: 1.0}
0: {-1: 0.0, 0: 0.0, 1: 1.0}
1: {-1: 0.5, 0: 0.5, 1: 0.0}
-------------------------------------------------------------------------------------------
Day 5: 
Decision: Y 
Portfolio Value: 0
Matrix:
-1: {-1: 0.0, 0: 0.0, 1: 1.0}
0: {-1: 0.0, 0: 0.5, 1: 0.5}
1: {-1: 0.5, 0: 0.5, 1: 0.0}
---------------------------------------------------------------------------------